## Pergunta: Coloque um cabeçalho nesse notebook com o seu nome e resumindo o que descobriu ao explorar esses dados

### <span style="color:red">  Aluno : Samuel Hericles Souza Silveira </span>
    
### <span style="color:red"> Data da realização: 07/06/2002 </span>

## Formação Data Science Alura - Análise exploratória do MovieLens

Na formação de Data Scienda www.alura.com.br um dos conjuntos de dados que utilizamos é o de avaliações para filmes no site MovieLens. Neste projeto faremos uma análise exploratória desses dados.

### Dicas para o projeto completo:

- todo gráfico deve ter um título, labels e legendas que fazem sentido
- configure um tamanho adequado para os gráficos
- utilize as versões dos arquivos de dados disponíveis no github

## Pergunta: conferindo as versões utilizadas

- Devemos usar pandas 0.24.0 ou mais recente
- Devemos usar seaborn 0.9.0 ou mais recente
- Devemos usar scipy 1.2.0 ou mais recente

In [452]:
# Caso não possua algumas dessas bibliotecas, descomente-a e rode esta célula.

# !pip install pandas=="0.24.0" --quiet
# !pip install seaborn=="0.9.0" --quiet
# !pip install scipy=="1.2.0" --quiet
# !pip install statsmodels
# !pip install matplotlib

In [453]:
import pandas as pd
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
from scipy.stats import shapiro 
from scipy.stats import  anderson
from scipy.stats import normaltest 

print("Usando pandas %s" % pd.__version__)
print("Usando seaborn %s" % sns.__version__)
print("Usando scipy %s" % scipy.__version__)

Usando pandas 1.0.4
Usando seaborn 0.9.0
Usando scipy 1.4.1


## Pergunta: carregue os filmes

- os dados `https://raw.githubusercontent.com/alura-cursos/formacao-data-science/master/movies.csv` devem ser carregado em `filmes`
- renomeie as colunas para `filmeId`, `titulo` e `generos`
- configure o indice (`index`) para ser o `filmeId`

In [454]:
# solução
filmes = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/formacao-data-science/master/movies.csv')
filmes.columns = ['filmeId','titulo','generos']
filmes.set_index('filmeId',inplace=True)

In [455]:
filmes.head()

,titulo,generos
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [456]:
print("Carregamos %d filmes" % len(filmes))
if(len(filmes) != 9742):
    print("ERRO! O conjunto de dados que estamos trabalhando nesse projeto possui 9742 elementos.")

Carregamos 9742 filmes


Vamos extrair o ano dos filmes (código já pronto): 

In [457]:
ano_de_lancamento = filmes.titulo.str.extract("\((\d{4})\)")
filmes['ano_de_lancamento'] = ano_de_lancamento.astype(float)
filmes.head()

,titulo,generos,ano_de_lancamento
filmeId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0
2,Jumanji (1995),Adventure|Children|Fantasy,1995.0
3,Grumpier Old Men (1995),Comedy|Romance,1995.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0
5,Father of the Bride Part II (1995),Comedy,1995.0


## Pergunta: carregando as notas

- os dados `https://raw.githubusercontent.com/alura-cursos/formacao-data-science/master/ratings.csv` devem ser carregado em `notas`
- renomeie as colunas para `usuarioId`, `filmeId`, `nota` e `momento`

In [458]:
# solução
notas = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/formacao-data-science/master/ratings.csv')
notas.columns = ['usuarioId','filmeId','nota','momento']

In [459]:
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Pergunta: gere uma nova feature, a média dos votos por filme

- crie uma nova coluna chamada `nota_media` dentro do dataframe `filmes`

Dicas:
- como filmeId já é o indice de seu dataframe filmes, se você possui uma série com o número de votos por filme, basta atribuir uma nova coluna normalmente que será usado o mesmo índice

In [460]:
# solução
filme_por_nota = notas.groupby('filmeId').nota
media_do_filme = filme_por_nota.mean()
filmes['nota_media'] = media_do_filme

<span style='color:blue'>
Organiza pelo o groupby o id dos filmes e capturando sua nota, após isso, cálcula a média dos grupos.
</span>

In [461]:
filmes.head()

,titulo,generos,ano_de_lancamento,nota_media
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,3.920930
2,Jumanji (1995),Adventure|Children|Fantasy,1995.0,3.431818
3,Grumpier Old Men (1995),Comedy|Romance,1995.0,3.259615
4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,2.357143
5,Father of the Bride Part II (1995),Comedy,1995.0,3.071429


## Pergunta: remova os filmes com menos de 50 avaliações

- conte o número de notas por filme
- adicione uma coluna nova chamada `total_de_votos` a `filmes` que representa o número de avaliações que aquele filme teve no conjunto de dados
- remova os que tem 49 ou menos de `filmes`

## Solução

In [462]:
# solução para calcular o total de votos por filme
total_de_votos = notas['filmeId'].value_counts()
filmes['total_de_votos'] = total_de_votos

<span style='color:blue'>
Conta a quantidade de notas dos filme, usando value_counts() ele organiza pelo index(filmeId) assim encotramos os valores.
</span>

In [463]:
# solução para criar a nova coluna e filtrar os filmes

index_filmes_menos_50 = filmes.query('total_de_votos < 50').index
index_filmes_menos_50
filmes.drop(index = index_filmes_menos_50,inplace=True)
filmes.dropna(inplace=True)

<span style='color:blue'>
Usando a função query() que retorna uma característica específica do dataframe, após isso organizamos os valores.
</span>

In [464]:
filmes.head()

,titulo,generos,ano_de_lancamento,nota_media,total_de_votos
filmeId,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,3.920930,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,1995.0,3.431818,110.0
3,Grumpier Old Men (1995),Comedy|Romance,1995.0,3.259615,52.0
6,Heat (1995),Action|Crime|Thriller,1995.0,3.946078,102.0
7,Sabrina (1995),Comedy|Romance,1995.0,3.185185,54.0


# Visualizando os dados gerais

## Pergunta: Visualize as notas médias

- histograma
- cummulative distribution function

Dicas:

- lembre-se de colocar um título nas imagens além de nomes nos eixos x e y
- para gerar a versão cumulativa lembre-se de usar os parâmetros `hist_kws` e `kde_kws` para definir `'cumulative'=True`

<span style='color:blue'>
Fiz esta célula pois não estava entedendo os valores no eixo y dos histogramas do matplotlib e seaborn. Logo, depois percebi que os valores lá contidos são a percetagem de dados que percentem a tal intervalo compreendido no eixo x.
</span>

In [ ]:
medias = filmes['nota_media'].round(2).value_counts().index.transpose()
freq_medias = filmes['nota_media'].round(2).value_counts().values.transpose()
colunas = {'Medias':medias,'Frequência':freq_medias}
medias_das_notas = pd.DataFrame(data=colunas)
medias_das_notas.set_index('Medias')    
sns.barplot(x="Frequência",y="Medias",data=medias_das_notas,color='y')

<span style='color:blue'>
Assim, as duas células abaixo são as requeridas pelo instrutor.
</span>

In [ ]:
# solução histograma
plt.figure(figsize=(8,5))
ax = sns.distplot(filmes['nota_media'],color='blue')
ax.set(xlabel = 'nota média', ylabel='Percetagens de filmes',title="Histograma - Média das notas dos filmes")
ax.grid()

<span style='color:blue'>
Este histograma está proximo a uma gaussina com valores de skewness perto de zero e curtose mesoocurtica.
</span>

In [ ]:
# solução cumulativa
kwargs = {'cumulative':True}
ax = sns.distplot(filmes['nota_media'],hist_kws=kwargs)
ax = sns.kdeplot(filmes['nota_media'],cumulative=True)
ax.set(xlabel = 'Nota média', ylabel='Percetagens de filmes(%)',title="Gráfico cumulativo - Média das notas dos filmes")
ax.grid()

<span style='color:blue'>
Não consegui retirar a linha em azul que compreende susposta gaussiana(veja nos testes mais abaixo) da função displot, essa linha refere-se ao histograma das notas médias dos filmes, como vista na célula acima.
</span>

## Pergunta: Visualize o total de votos

- histograma
- cummulative distribution function

In [ ]:
# solução histograma
ax = sns.distplot(filmes['total_de_votos'],color='blue')
ax.set(xlabel = 'Nota média', ylabel='% do total de votos',title="Histograma - Total de votos dos filmes")
ax.grid()

<span style='color:blue'>
Este histograma está proximo a uma gaussina com skewness>0 e curtose leptocurtica.
</span>

In [ ]:
# solução cumulativa
kwargs = {'cumulative':True}
ax = sns.distplot(filmes['total_de_votos'],hist_kws=kwargs)
ax = sns.kdeplot(filmes['total_de_votos'],cumulative=True)
ax.set(xlabel = 'Total de votos', ylabel='% de filmes',title="Gráfico cumulativo - Total de votos dos filmes")
ax.grid()

## Pergunta: 20% dos filmes do dataframe tem nota maior que? 

- lembre-se da função `quantile` do pandas

In [ ]:
# solução
nota_limite_dos_20_porcento = filmes['nota_media'].quantile(0.2)
nota_limite_dos_20_porcento

<span style='color:blue'>
A minha duvída aqui é que este 20% são dos piores filmes ou dos melhores filmes, o que fiz na célula acima foi dos piores filmes pegando todo o conjunto de dados.
</span>

In [ ]:
print("20 porcento dos filmes tem nota maior que de %.2f" % nota_limite_dos_20_porcento)

<span style='color:blue'>
Aqui abaixo mostro a nota média dos 20% dos melhores filmes.
</span>

In [ ]:
nota_limite_dos_80_porcento = filmes['nota_media'].quantile(0.8)
nota_limite_dos_80_porcento
print("20 porcento dos melhores filmes tem nota maior que de %.2f" % nota_limite_dos_80_porcento)

## Vamos separar a coluna `generos` em diversas colunas.

- O código já está pronto.

In [ ]:
filmes['generos'] = filmes['generos'].str.replace('-', '')
generos = filmes['generos'].str.get_dummies()
print("Temos %d generos distintos" % generos.shape[1])

In [ ]:
filmes = filmes.join(generos).drop(columns=['generos'], axis=1)
filmes.head()

## Pergunta: visualização de quantos filmes existem por genero

- Para cada um dos generos você deve agrupar o número de filmes
- Ordene as barras, na esquerda o genero com mais filmes, na direita o que tem menos filmes

Dica:

- Para fazer a soma de uma ou mais colunas individualmente você pode usar `seu_dataframe.sum()`
- Não precisa usar o dataframe `filmes`, somente o dataframe `generos`
- Em certas situações é mais fácil usar as funções de plotagem do pandas ao invés do seaborn

In [ ]:
# solução
genero = ['Action','Adventure', 'Animation', 'Children',
           'Comedy', 'Crime', 'Documentary', 'Drama',
           'Fantasy', 'FilmNoir', 'Horror', 'IMAX',
           'Musical', 'Mystery','Romance', 'SciFi',
           'Thriller', 'War', 'Western']
quantidade_de_filmes_por_genero = []
for i in genero:
    quantidade_de_filmes_por_genero.append(filmes[i].sum())
df_freq_filmes = pd.DataFrame({'Genero':genero,
                               'Frequencia':quantidade_de_filmes_por_genero})
df_popularidade = df_freq_filmes
df_freq_filmes = df_freq_filmes.sort_values('Frequencia',ascending=False)
df_freq_filmes.reset_index(drop=True, inplace=True)
df_freq_filmes.head()

<span style='color:blue'>
Pega todos os generos de filmes, a partir deles soma quantidade de vezes que que sua colunas possui valores 1's e insere no dataframe.
</span>

In [ ]:
plt.figure(figsize=(20,8))
ax = sns.barplot(x="Genero",y="Frequencia",data=df_freq_filmes,palette='GnBu_d')
ax.set_xlabel('Gêneros',fontsize=20)
ax.set_ylabel('Frequêncas',fontsize=20)
ax.set_title('Frequências de filmes por gênero',fontsize=30)

<span style='color:blue'>
Vemos que filmes de drama e ação são bem proximos, além de filmes de crime e romance. Outra coisa a se notar, que se não todos, mas a maioria dos filmes possuem mais de duas categorias, logo duas ou mais categorias podem está presentes em filmes o que pode mostrar que alguns tipos de filmes podem ter contagens bem próximas.
</span>

## Pergunta: Quais são os 5 generos com mais filmes?

Dica:

- `sort_values()` pode ordenar uma Series do pandas

In [ ]:
# solução
top5_generos_em_quantidade_de_filmes = df_freq_filmes.head()

In [ ]:
print(top5_generos_em_quantidade_de_filmes)

## Pergunta: visualização de votos por genero

- o código que gera os votos por genero já está pronto
- ordene as categorias da mais visualizada para a menos visualizada
- gere a visualização por genero

In [ ]:
votos_por_genero_por_filme = pd.DataFrame((generos.values.T * filmes['total_de_votos'].values).T, columns = generos.columns)
votos_por_genero_por_filme.head()

In [ ]:
# solucão
total_de_votos = []
for i in votos_por_genero_por_filme.columns:
    total_de_votos.append(votos_por_genero_por_filme[i].sum())
df_popularidade['total_de_votos'] = total_de_votos
total_de_votos

<span style='color:blue'>
Soma a quantidade de votos por categoria e organiza o vetor em ordem descrecente.
</span>

In [ ]:
# solução
qt_de_votos_por_generos = pd.DataFrame({'Genero':votos_por_genero_por_filme.columns,
                                        'Total de votos':total_de_votos})
qt_de_votos_por_generos.reset_index(drop=True, inplace=True)
qt_de_votos_por_generos.sort_values('Total de votos',ascending=False,inplace=True)
qt_de_votos_por_generos.head()

<span style='color:blue'>
Coloca os resultados em um dataframe para facilitar a visualização do ranking.
</span>

In [ ]:
plt.figure(figsize=(20,8))
ax = sns.barplot(x="Genero",y="Total de votos",data=qt_de_votos_por_generos,palette='GnBu_d')
ax.set_xlabel('Gêneros',fontsize=20)
ax.set_ylabel('Quantidade de votos',fontsize=20)
ax.set_title('Quantidade de votos por gênero',fontsize=30)

<span style='color:blue'>
Vemos que filmes de drama e comédia são bem proximos, além de filmes de crime e SciFi. Outra coisa a se notar, que se não todos, mas a maioria dos filmes possuem mais de duas categorias, logo duas ou mais categorias podem está presentes em filmes o que pode mostrar que alguns tipos de filmes podem ter contagens bem próximas. Sendo assim, se um usuário votou em uma categoria pode está também votando em outra(as) junta.
</span>

## Pergunta : votos por filme por genero

Queremos entender a quantidade de votos em relação a quantidade de filmes existentes naquele genero.

Por exemplo: 
- se temos 1000 votos de `Ação` e 10 filmes desse genero, foram 100 votos por filme nesse genero.
- se o gênero `Documentário` tem 2 filmes e 500 votos, ele tem 250 votos por filme nesse gênero.

Queremos entender isso, a "popularidade" média de um filme em cada categoria, isto é, o número de votos dividido pelo número de filmes, organizado para cada categoria.

- Você já possui o número de votos por genero
- Calcule o número de votos por genero dividido pelo total de filmes por genero
- Ordene do maior para o menor
- Plote um gráfico para visualizarmos esses dados

In [ ]:
# solucao código
df_popularidade['Voto/Filme'] = df_popularidade['total_de_votos']/df_popularidade['Frequencia']
df_popularidade.sort_values('Voto/Filme',ascending=False, inplace=True)

In [ ]:
df_popularidade

In [ ]:
# solução com visualização 
plt.figure(figsize=(20,8))
ax = sns.barplot(x="Genero",y="Voto/Filme",data=df_popularidade,palette='GnBu_d')
ax.set_xlabel('Gêneros',fontsize=20)
ax.set_ylabel('Voto/Filme',fontsize=20)
ax.set_title('Votos por filme por genero',fontsize=30)

<span style='color:blue'>
A categoria War tem maior valor de Voto/Filme pois a mesma tem apenas 19 filmes mas 2113 votos, ou seja, poucos filmes e muitos votos. Além disso, pode-se perceber um gosto dos usuários por filmes desse tipo em comparação de filmes de Western(velho oeste) que 12 filmes mas apenas 900 votos.
    
Contundo, pode-se também levar em questão sobre o argumentos de filmes com mais de uma categoria que filmes de guerra pode se correlacionar bem com todos os outros tipos de filmes.
</span>

## Pergunta: Olhando as 3 últimas visualizações o que você conclui sobre os generos `Action` e `War`?

In [ ]:
df_popularidade

### Solução (texto livre):
    
<span style='color:red'>
    A nota do gênero Action é mais confiavel, pois houve mais pessoas vontado nesse estilo de filme. Logo, ocorre o contrário no genêro War porque ele recebe o valor valor em Voto/Filmes, mas só há 19 filmes e mais de 2000 votos. Além disso, vemos que os usuários gostam mais de filmes de ação, por conta que há mais filmes nessa categoria.
</span>

    

## Pergunta: Crie 2 boxplots no mesmo plot comparando a coluna `total_de_votos` dos filmes de categoria `Horror` e `Adventure`  

Dica:

- as vezes é mais fácil usar o boxplot do matplotlib ao invés do seaborn
- plt.boxplot suporta o parâmetro labels com uma lista de nomes para cada boxplot plotado

In [ ]:
# solução:
plt.boxplot([votos_por_genero_por_filme['Horror'],votos_por_genero_por_filme['Adventure']])
plt.xticks([1, 2], ['Horror','Adventure'])
plt.title('Boxplot dos filmes de horror e aventura',fontsize=16)
plt.xlabel('Categoria de filme',fontsize=14)
plt.ylabel('Quantidade de votos',fontsize=14)


<span style='color:blue'>
Este boxplot não mostra muita coisa, pois há uma enorme quantidade de valores nulos isso faz com as medidas de tedência central(média, mediana, moda, quartis,etc) que dependem da quantidade de amostra para dar um resultado mostram valores errados. Então, se retirarmos esses valores, podemos ver algo mais atraente para conclusões.
</span>

In [ ]:
Horror = votos_por_genero_por_filme['Horror']
Aventura = votos_por_genero_por_filme['Adventure']
plt.boxplot([Horror[Horror>0],Aventura[Aventura>0]])
plt.xticks([1, 2], ['Horror','Adventure'])
plt.title('Boxplot dos filmes de horror e aventura',fontsize=16)
plt.xlabel('Categoria de filme',fontsize=14)
plt.ylabel('Quantidade de votos',fontsize=14)

## Pergunta: mostre agora um boxplot para cada uma das categorias

Dica:
- use generos.columns para acessar todos os gêneros
- já removemos os hífens dos nomes das colunas quando criamos as colunas de gêneros
- plt (pyplot) do matplotlib suporta a função xticks(rotation=90) para rotacionar os labels

In [ ]:
plt.figure(figsize=(18,8))
plt.boxplot(votos_por_genero_por_filme[generos.columns.values].transpose())
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], generos.columns.values)
plt.title('Boxplot de tipos de filmes - sem a remoção do zeros',fontsize=16)
plt.xlabel('Categoria de filme',fontsize=14)
plt.ylabel('Quantidade de votos',fontsize=14)
plt.grid()

<span style='color:blue'>
Este boxplot não mostra muita coisa, pois há uma enorme quantidade de valores nulos isso faz com as medidas de tedência central(média, mediana, moda, quartis,etc) que dependem da quantidade de amostra para dar um resultado mostram valores errados. Então, se retirarmos esses valores, podemos ver algo mais atraente para conclusões.
</span>

In [ ]:
a = []
for i in generos.columns.values:
    a.append(votos_por_genero_por_filme[i].values)
b = []
for i in range(1,19):
    b.append(a[i][a[i]>0].transpose())

plt.figure(figsize=(18,8))
plt.boxplot(b)
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], generos.columns.values)
plt.title('Boxplot de tipos de filmes - com a remoção do zeros',fontsize=16)
plt.xlabel('Categoria de filme',fontsize=14)
plt.ylabel('Quantidade de votos',fontsize=14)
plt.grid()

## Pergunta: desejo criar um filme com somente um dos gêneros: `Horror` ou `Adventure`. Quero ter uma chance maior de ter um faturamento que pague minhas contas, mesmo que não seja um super sucesso absurdo de vendas. Qual dos 2 gêneros você sugere?

In [ ]:
Aventura = votos_por_genero_por_filme['Adventure']
Horror = votos_por_genero_por_filme['Horror']

plt.boxplot([Aventura[Aventura>0],Horror[Horror>0]])
plt.xticks([1,2], ['Aventura','Horror'])
plt.title('Boxplot de filmes de Horror e aventura',fontsize=16)
plt.xlabel('Categoria de filme',fontsize=14)
plt.ylabel('Quantidade de votos',fontsize=14)
plt.grid()

### Solução:

<span style='color:red'>
    Cria um filme de aventura! pois há um público maior nele(com base no total de votos), e você tem uma maior chance de obter uma boa nota conforme é visto no boxplot acima. Além disso,a mediana é mais proxima da média e a maior parte dos filmes são muito votados, ou seja, tem mais chance de serem vistos.

</span>


## Pergunta: nossa análise foi até então visual. Aplique um teste estatístico para verificar se existe significância estatística na distribuição dos votos dos filmes de `Adventure` e `Horror`. Sua recomendação continua a mesma? Explique.

Passos:

- visualize a distribuição de ambas através de histogramas
- se suspeitar normalidade na distribuição, faça um teste de normalidade para ambas
- aplique um teste adequado de comparação das distribuições
- explique o resultado encontrado
- utilize p=0.05

In [ ]:
# solucao histograma de aventura

# Retirando os zeros no conjunto temos:
plt.figure(figsize=(8,4))
ax = Aventura[Aventura>0].hist()
ax = plt.title('Histograma do números de votos de filmes de aventura',fontsize=14)
ax = plt.xlabel('Número de votos',fontsize=14)
ax = plt.ylabel('Quantidade de filmes',fontsize=14)

<span style='color:blue'>
Aqui podemos notar que pode parecer com uma distibuição gaussiana com skewness>0 e curtoe leptocurtica, mas para ter certeza faremos abaixo os testes estatítiscos.
</span>

In [ ]:
# solucao histograma de horror

# Retirando os zeros no conjunto temos:
plt.figure(figsize=(8,4))
Horror[Horror>0].hist()
ax = plt.title('Histograma do números de votos de filmes de horror',fontsize=14)
ax = plt.xlabel('Número de votos',fontsize=14)
ax = plt.ylabel('Quantidade de filmes',fontsize=14)

<span style='color:blue'>
Aqui podemos notar que  não parece com uma distibuição gaussiana, mas para ter certeza faremos abaixo os testes estatítiscos.
</span>

In [ ]:
# solucao teste de normalidade ou justificativa para nao utiliza-lo

# Teste shapiro-wilk avalia se as amostra se comportam em uma gaussiana, boa para um pequeno conjunto de amostras.
def teste_shapiro(dados):
    stat,p = shapiro(dados)
    alfa  =  0.05 
    if  p  >  alfa : 
        msg  =  'Teste de shapiro - A amostra parece gaussiana (falha ao rejeitar H0)' 
    else : 
        msg  =  'Teste de shapiro - A amostra não parece gaussiana (rejeita H0)'
    print(msg)
    return stat,p

# Teste de anderson, que avalia qual tipo de distribuição os dados se comportam, retorna vários valores além do valor p.
def teste_anderson(dados):
    result = anderson(dados)
    stat = round(result.statistic, 4)
    p = 0
    result_mat = []
    for i in range(len(result.critical_values)):
        sl, cv = result.significance_level[i], result.critical_values[i]
        if result.statistic < result.critical_values[i]:
            msg = 'Teste de anderson - A amostra parece gaussiana (não rejeita H0)'
        else:
            msg = 'Teste de anderson - A amostra não parece gaussiana (rejeita H0)'
        result_mat.append([len(dados), stat, sl, cv, msg])
    print(msg)

# Teste D'Agostino ou K² teste, retorna um remuso estatístico dos dados.
def test_dagostino(dados):
    stat, p = normaltest(dados)
    alpha = 0.05
    if p > alpha:
        msg = "Teste de D'Agostino - A amostra parece gaussiana (não rejeita H0)"
    else:
        msg = "Teste de D'Agostino - A amostra não parece gaussiana (rejeita H0)"
    print(msg)

In [ ]:
# solução com o teste desejado

teste_shapiro(Aventura[Aventura>0])
teste_anderson(Aventura[Aventura>0])
test_dagostino(Aventura[Aventura>0])
print("*" * 80)
teste_shapiro(Horror[Horror>0])
teste_anderson(Horror[Horror>0])
test_dagostino(Horror[Horror>0])

### Solução (explique sua conclusão):
<span style='color:red'>

Não foi necessário realizar os testes para saber que as duas distribuições não são gaussiana, mas utilizei da matemática para ter como aliada. Logo, esses dados foram retirados a partir de uma coleta de dados com viés, que podemos citar a preferência dos usuários por um certa categoria de filmes ou o número de filmes disponveis em um determinado tipo.
Portanto, minha sugestão ainda fica a mesma e também posso adicionar que os filmes de aventura tem mais chances de ser um preferências dos usuários da plataforma.
    
</span>



## Pergunta: Faça os mesmos boxplots para as notas médias (`nota_media`) dos gêneros `Adventure` e `Horror`.

In [ ]:
# solução:
nota_media_horror = filmes.query('Horror==1').nota_media
nota_media_aventura = filmes.query('Adventure==1').nota_media


In [ ]:
plt.boxplot([nota_media_aventura,nota_media_horror])
plt.xticks([1,2], ['Aventura','Horror'])
plt.title('Boxplot de filmes de Horror e aventura',fontsize=16)
plt.xlabel('Categoria de filme',fontsize=14)
plt.ylabel('Nota média',fontsize=14)
plt.grid()

## Pergunta: Quero fazer um filme que seja sucesso de crítica, com pessoas dando notas altas, mesmo que não tenha um número muito grande de pessoas assistindo. Qual gênero você recomenda se tenho que escolher somente um entre `Adventure` e `Horror`?

### Solução:
<span style='color:red'>
Horror, pois há uma maior chance do que receber notas mais altas com filmes desse tipo. Veja no boxplot acima, a distribuição da categoria horror é mais densa nos intervalos [3.4, 4.0] e os filmes de aventura no [3.4, 3.8].
</span>

## Pergunta: nossa análise foi até então visual. Aplique um teste estatístico para verificar se existe significância estatística na distribuição das notas dos filmes de `Adventure` e `Horror`.

Passos:

- visualize a distribuição de ambas através de histogramas
- se suspeitar normalidade na distribuição, faça um teste de normalidade para ambas
- aplique um teste adequado de comparação das distribuições
- explique o resultado encontrado
- utilize p=0.05

In [ ]:
# solucao histograma de aventura
plt.figure(figsize=(8,4))
nota_media_aventura.hist()
ax = plt.title('Histograma das notas médias de filmes de aventura',fontsize=14)
ax = plt.xlabel('Notas médias',fontsize=14)
ax = plt.ylabel('Quantidade de filmes',fontsize=14)

<span style='color:blue'>
Aqui podemos notar que pode parecer com uma distibuição gaussiana com skewness negativa e curtose mesocurtica, mas para ter certeza faremos abaixo os testes estatítiscos.
</span>

In [ ]:
# solucao histograma de horror
plt.figure(figsize=(8,4))
nota_media_horror.hist()
ax = plt.title('Histograma das notas médias de filmes de horror',fontsize=14)
ax = plt.xlabel('Notas médias',fontsize=14)
ax = plt.ylabel('Quantidade de filmes',fontsize=14)

<span style='color:blue'>
Aqui podemos notar que pode parecer com uma distibuição gaussiana com skewness>0 e curtoe leptocurtica, mas para ter certeza faremos abaixo os testes estatítiscos.
</span>

In [ ]:
# solucao teste de normalidade ou justificativa para nao utiliza-lo
teste_shapiro(nota_media_horror)
teste_anderson(nota_media_horror)
test_dagostino(nota_media_horror)
print("*" * 80)
teste_shapiro(nota_media_aventura)
teste_anderson(nota_media_aventura)
test_dagostino(nota_media_aventura)

### Solução com o teste desejado => Explicação dos resultados
<span style='color:red'>
    Não foi necessário realizar os testes para saber que as duas distribuições não são gaussiana, mas utilizei da matemática para ter como aliada. Logo, esses dados foram retirados a partir de uma coleta de dados com viés, que podemos citar a preferência dos usuários por um certa categoria de filmes ou o número de filmes disponveis em um determinado tipo.
</span>

## Sua recomendação para maior sucesso de crítica em nota alta entre `Horror` e `Adventure` continua a mesma? Explique.



### Solução:
<span style='color:red'>
    Não, pois vendo que a categoria de filmes Adventure se mostrou que suas amostras se comportam como uma gaussiana em dois dos três testes. Além disso, segundo o histograma do filme de aventuras, vemos que boa parte dos filmes esta no intervalo [3.4, 4.0].
</span>

## Pergunta: Quero fazer um filme que seja primeiro sucesso de crítica, com pessoas dando notas altas. Em segundo lugar quero que seja popular (mais pessoas assistam). Com as informações visuais e estatísticas, supondo poder escolher somente um, qual dois 2 gêneros você recomenda agora? `Horror` ou `Adventure`?

### Solução:
<span style='color:red'>
    Pelos os testes anteriores, vemos que o mais aconselhado é fazer filmes de aventura(Adventure).
</span>

## Pergunta: Quais são os top 5 filmes dos 2 generos mais assistidas?

- utilize os 2 generos mais votados em `votos_por_genero`
- utilize a `nota_media` para definir o top 5 filmes
- o resultado são 5, não 10 filmes

In [ ]:
aux = df_popularidade.sort_values('total_de_votos',ascending=False)
votos_por_genero = aux.iloc[[0,1]]

In [ ]:
# solução: calculando os 2 generos mais votados
votos_por_genero

In [ ]:
# solução: encontrando os top 5 filmes desses generos
top5_action = filmes.query('Action == 1').sort_values('nota_media',ascending=False)
top5_action.iloc[[0,1,2,3,4]]

In [ ]:
top5_drama = filmes.query('Drama == 1').sort_values('nota_media',ascending=False)
top5_drama.iloc[[0,1,2,3,4]]

## Pergunta: Quais são os 2 generos com maior valor de correlação com a `nota_media`? E os 2 com menor valor de correlação com a `nota_media`?

- O pandas possui a função `corr` para calcular correlação entre todas as colunas

In [ ]:
aux = filmes.corr().sort_values('nota_media',ascending=False)['nota_media']
aux

In [ ]:
# solucao
print("As duas categorias com maiores correlação são Drama({}) e Crime({})".format(aux[2].round(2),aux[3].round(2)))
print("*"*80)
print("As duas categorias com Menores correlação são Documentário({}) e Musical({})".format(aux[9].round(3),aux[10].round(2)))

## Pergunta: Quais são os 2 generos com maior valor de correlação com `total_de_votos`? E quais os 2 com menor valor de correlação com `total_de_votos`?

In [ ]:
aux = filmes.corr().sort_values('total_de_votos',ascending=False)['total_de_votos']
aux

In [ ]:
# solucao
print("As duas categorias com maiores correlação são Aventura({}) e Guerra({})".format(aux[2].round(2),aux[3].round(2)))
print("*"*80)
print("As duas categorias com Menores correlação são Infantil({}) e Mistério({})".format(aux[10].round(3),aux[11].round(3)))

## Pergunta: Visualize graficamente o relacionamento entre ano de lançamento e nota média

- Use o gráfico `lineplot`

In [ ]:
# solucao
total_de_votos_e_ano_lan_values = filmes[['nota_media','ano_de_lancamento']]
plt.figure(figsize=(20,8))
ax = sns.lineplot(x='ano_de_lancamento',y='nota_media',data=total_de_votos_e_ano_lan_values)
ax.set_xlabel('Ano de lançamento',fontsize=20)
ax.set_ylabel('Nota média',fontsize=20)
ax.set_title('Gráfico que mostra a correlação entre nota média e ana de lançamento',fontsize=20)

## Pergunta: Interprete a correlação entre `ano_de_lancamento` e a `nota_media`?

- Calcule a correlação numericamente (use o `corr` do pandas)
- Correlação não implica causalidade, qual "problema" na coleta dos dados pode gerar esse tipo de correlação?

In [ ]:
# solução (código)
corr_total_de_votos_e_ano_lan_values = filmes[['nota_media','ano_de_lancamento']].corr()
corr_total_de_votos_e_ano_lan_values

In [ ]:
plt.figure(figsize=(20,6))
ax = sns.scatterplot(x='ano_de_lancamento',y='nota_media',data=total_de_votos_e_ano_lan_values)
ax.set_xlabel('Ano de lançamento',fontsize=20)
ax.set_ylabel('Nota média',fontsize=20)
ax.set_title('Gráfico que mostra a correlação entre nota média e ana de lançamento',fontsize=20)

### Solução (explicação)
<span style='color:red'>
    O que podemos ver que a há mais filmes entre as decadas de 1990 e 2010 e isso causa o baixo valor de correlação(próximo a zero).
</span>

In [ ]:
ano_lan = []
media = []
for i in range(len(filmes[['nota_media','ano_de_lancamento']])):
    if filmes[['nota_media','ano_de_lancamento']].iloc[i].values[1] > 1994:
        ano_lan.append(filmes['ano_de_lancamento'].iloc[i])
        media.append(filmes['nota_media'].iloc[i])
decada_de_90_pra_frente = pd.DataFrame({'Ano_lan':ano_lan,'Media':media})

plt.figure(figsize=(20,6))
ax = sns.scatterplot(x="Ano_lan",y="Media",data=decada_de_90_pra_frente)
ax = sns.lineplot(x="Ano_lan",y="Media",data=decada_de_90_pra_frente,color='red')
ax.set_xlabel('Ano de lançamento',fontsize=14)
ax.set_ylabel('Nota média',fontsize=14)
ax.set_title('Correlação de nota média com ano de lançamento pós década de 90',fontsize=14)

In [ ]:
decada_de_90_pra_frente.corr()

In [ ]:
ano_lan = []
media = []
for i in range(len(filmes[['nota_media','ano_de_lancamento']])):
    if ~(filmes[['nota_media','ano_de_lancamento']].iloc[i].values[1] > 1994):    
        ano_lan.append(filmes['ano_de_lancamento'].iloc[i])
        media.append(filmes['nota_media'].iloc[i])
        
decada_de_90_pra_tras = pd.DataFrame({'Ano_lan':ano_lan,'Media':media})
plt.figure(figsize=(20,6))
ax = sns.scatterplot(x="Ano_lan",y="Media",data=decada_de_90_pra_tras)
ax = sns.lineplot(x="Ano_lan",y="Media",data=decada_de_90_pra_tras,color='red')
ax.set_xlabel('Ano de lançamento',fontsize=14)
ax.set_ylabel('Nota média',fontsize=14)
ax.set_title('Correlação de nota média com ano de lançamento antes da década de 90',fontsize=14)

In [ ]:
decada_de_90_pra_tras.corr()

<span style='color:blue'>
Observe que já há um valor positivo e dobro do módulo do valor da correlação anterior.
</span>

## Pergunta: analise um pairplot

- crie uma variável chamada `infos` baseada nas colunas `ano_de_lancamento`, `nota_media` e `total_de_votos` de `filmes`
- faça o pairplot usando seaborn
- coloque uma regressão linear para cada pequeno plot dentro dele

In [ ]:
# solução
infos = filmes[['ano_de_lancamento','nota_media','total_de_votos']]
ax = sns.pairplot(infos,kind='reg')

## Pergunta: analise um pairplot dos filmes de `Animation`

- use o dataframe `filmes`
- em um único pairplot separe com cores distintas os filmes da categoria `Animation` dos outros
- use uma palete como `husl`
- use marcadores distintos para cada um dos dois grupos
- faça o pairplot usando seaborn e coloque uma regressão linear para cada pequeno plot dentro dele
- tanto em x quanto y use somente três variáveis: `total_de_votos`, `ano_de_lancamento`, `nota_media`

In [ ]:
infos = filmes[['ano_de_lancamento','nota_media','total_de_votos','Animation']]
infos

In [ ]:
plt.figure(figsize=(20,8))
ax = sns.pairplot(infos,kind='reg',hue='Animation',palette='husl'
                  ,vars=['ano_de_lancamento','nota_media','total_de_votos']
                 ,hue_order=[0,1])

## Pergunta: Analise a regressão em relação ao ano de lançamento.

- Como parece se comportar a `nota_media` de filmes de `Animation` mais antigos e mais recentes? (ano_de_lancamento x nota_media). Essa correlação parece ser baixa, mas parece ser linear?
- E o `ano_de_lancamento` x `total_de_votos`?
- E das outras categorias? Compare isso com os outros filmes.
- O que leva você levantar como hipótese para esse resultado?

### Solução:



#### Como parece se comportar a `nota_media `de filmes de Animation mais antigos e mais recentes? (`ano_de_lancamento x nota_media`). Essa correlação parece ser baixa, mas parece ser linear?    

In [ ]:
ano_lan_sim = []
total_votos_sim = []
ano_lan_nao = []
total_votos_nao = []

for i in range(len(filmes[['nota_media','ano_de_lancamento']])):
    if filmes[['nota_media','ano_de_lancamento']].iloc[i].values[1] > 1990:
        ano_lan_sim.append(filmes['ano_de_lancamento'].iloc[i])
        total_votos_sim.append(filmes['nota_media'].iloc[i])
    else:
        ano_lan_nao.append(filmes['ano_de_lancamento'].iloc[i])
        total_votos_nao.append(filmes['nota_media'].iloc[i])
        
decada_de_90_pra_frente = pd.DataFrame({'Ano_lancamento':ano_lan_sim,'nota_media':total_votos_sim})
decada_de_90_pra_tras = pd.DataFrame({'Ano_lancamento':ano_lan_nao,'nota_media':total_votos_nao})


plt.figure(figsize=(15,14))
plt.subplot(211)
ax1 = sns.scatterplot(x="Ano_lancamento",y="nota_media",data=decada_de_90_pra_frente)
ax1 = sns.lineplot(x="Ano_lancamento",y="nota_media",data=decada_de_90_pra_frente,color='red')
corr_depois_decada_1990 = decada_de_90_pra_frente.corr()['nota_media'].values[0].round(3)
ax1.set_xlabel('Ano de lançamento', fontsize=12)
ax1.set_ylabel('Nota média', fontsize=12)
ax1.set_title('Ano de lançamento x nota média, depois da década de 1990 correlação de %.3f' % 
              corr_depois_decada_1990,fontsize=18)

plt.subplot(212)
ax2 = sns.scatterplot(x="Ano_lancamento",y="nota_media",data=decada_de_90_pra_tras)
ax2 = sns.lineplot(x="Ano_lancamento",y="nota_media",data=decada_de_90_pra_tras,color='red')
corr_antes_decada_1990 = decada_de_90_pra_tras.corr()['nota_media'].values[0].round(3)
ax2.set_xlabel('Ano de lançamento', fontsize=12)
ax2.set_ylabel('Nota média', fontsize=12)
ax2.set_title('Ano de lançamento x nota média, antes da década de 1990 correlação de %.3f' % 
              corr_antes_decada_1990,fontsize=18)


<span style='color:blue'>
Quando se plota todos os valores vemos que os anos mais passados possuem pouca quantidade de filmes, logo podemos ver uma **correlação negativa** entre **ano_de_lancamento x nota_media** já se pegamos os dados a partir da **década de 1990** vemos que as notas médias possuem **correlação positiva** com o ano de lançamento do filme.
</span>

#### E o ano_de_lancamento x total_de_votos?

In [ ]:
ano_lan_sim = []
total_votos_sim = []
ano_lan_nao = []
total_votos_nao = []

for i in range(len(filmes[['total_de_votos','ano_de_lancamento']])):
    if filmes[['total_de_votos','ano_de_lancamento']].iloc[i].values[1] > 1990:
        ano_lan_sim.append(filmes['ano_de_lancamento'].iloc[i])
        total_votos_sim.append(filmes['total_de_votos'].iloc[i])
    else:
        ano_lan_nao.append(filmes['ano_de_lancamento'].iloc[i])
        total_votos_nao.append(filmes['total_de_votos'].iloc[i])
        
decada_de_90_pra_frente = pd.DataFrame({'Ano_lancamento':ano_lan_sim,'total_de_votos':total_votos_sim})
decada_de_90_pra_tras = pd.DataFrame({'Ano_lancamento':ano_lan_nao,'total_de_votos':total_votos_nao})


plt.figure(figsize=(15,14))
plt.subplot(211)
ax1 = sns.scatterplot(x="Ano_lancamento",y="total_de_votos",data=decada_de_90_pra_frente)
ax1 = sns.lineplot(x="Ano_lancamento",y="total_de_votos",data=decada_de_90_pra_frente,color='red')
corr_depois_decada_1990 = decada_de_90_pra_frente.corr()['total_de_votos'].values[0].round(3)
ax1.set_xlabel('Ano de lançamento', fontsize=12)
ax1.set_ylabel('Total de votos', fontsize=12)
ax1.set_title('Ano de lançamento x total de votos, depois da década de 1990 correlação de %.3f' % 
              corr_depois_decada_1990,fontsize=18)

plt.subplot(212)
ax2 = sns.scatterplot(x="Ano_lancamento",y="total_de_votos",data=decada_de_90_pra_tras)
ax2 = sns.lineplot(x="Ano_lancamento",y="total_de_votos",data=decada_de_90_pra_tras,color='red')
corr_antes_decada_1990 = decada_de_90_pra_tras.corr()['total_de_votos'].values[0].round(3)
ax2.set_xlabel('Ano de lançamento', fontsize=12)
ax2.set_ylabel('Total de votos', fontsize=12)
ax2.set_title('Ano de lançamento x total de votos, antes da década de 1990 correlação de %.3f' % 
              corr_antes_decada_1990,fontsize=18)


<span style='color:blue'>
Quando se plota todos os valores vemos que os anos mais passados possuem pouca quantidade de filmes, logo podemos ver uma **correlação positiva** entre **ano_de_lancamento x total_de_votos** já se pegarmos os dados a partir da **década de 1990** vemos que as notas médias possuem **correlação negativa** com o ano de lançamento do filme.
</span>


<span style="color:red">**Creio que esse gráficos já respondem as outras perguntas, pois estou analisando todos os filmes, logo caso colocassem apenas algumas categorias poderia chegar a conclusões erradas devido alguns possuirem poucos dados**</span>.

## Pergunta: Explore

- compartilhe conosco 2 visualizações extras que julgar interessante
- explique o que encontrou
- use quantas células (texto ou código) julgar necessário

### Solução
<span style='color:red'>
Primeiro, são os graficos da questão anterior, algo ocorreu que a partir da década de 1990 (pelo gráfico, em 1994 especificamente) aumentasse o número de filmes.
</span>

# <span style="color:blue" >Análise das 5 melhores categorias</span>

In [ ]:
df_popularidade.sort_values('Frequencia',ascending=False).head()

<span style='color:blue'>
Os filmes de comédia são os pior avaliado dentre as 5 melhores categorias
</span>

In [ ]:
def decada_de_90(dados): 
    ano_lan_sim = []
    total_votos_sim = []
    ano_lan_nao = []
    total_votos_nao = []

    for i in range(len(dados[['total_de_votos','ano_de_lancamento']])):
        if dados[['total_de_votos','ano_de_lancamento']].iloc[i].values[1] > 1990:
            ano_lan_sim.append(dados['ano_de_lancamento'].iloc[i])
            total_votos_sim.append(dados['total_de_votos'].iloc[i])
        else:
            ano_lan_nao.append(dados['ano_de_lancamento'].iloc[i])
            total_votos_nao.append(dados['total_de_votos'].iloc[i])

    decada_de_90_pra_frente = pd.DataFrame({'Ano_lancamento':ano_lan_sim,'total_de_votos':total_votos_sim})
    decada_de_90_pra_tras = pd.DataFrame({'Ano_lancamento':ano_lan_nao,'total_de_votos':total_votos_nao})
    return decada_de_90_pra_frente,decada_de_90_pra_tras

In [ ]:
filmes_de_comedia  = filmes[['titulo','ano_de_lancamento','nota_media','total_de_votos','Comedy']].query('Comedy==1')
filmes_de_drama    = filmes[['titulo','ano_de_lancamento','nota_media','total_de_votos','Drama']].query('Drama==1')
filmes_de_acao     = filmes[['titulo','ano_de_lancamento','nota_media','total_de_votos','Action']].query('Action==1')
filmes_de_aventura = filmes[['titulo','ano_de_lancamento','nota_media','total_de_votos','Adventure']].query('Adventure==1')
filmes_de_terror   = filmes[['titulo','ano_de_lancamento','nota_media','total_de_votos','Thriller']].query('Thriller==1')

In [ ]:
comedia_depois_90,comedia_antes_90 = decada_de_90(filmes_de_comedia)
drama_depois_90,drama_antes_90 = decada_de_90(filmes_de_drama)
acao_depois_90,acao_antes_90 = decada_de_90(filmes_de_acao)
aventura_depois_90,aventura_antes_90 = decada_de_90(filmes_de_aventura)
terror_depois_90,terror_antes_90 = decada_de_90(filmes_de_terror)

In [ ]:
rank_de_votos_depois_90 = pd.DataFrame({'Categoria':['Comédia','Drama','Ação','Aventura','Terror'],
                                       'Total_de_votos':[comedia_depois_90['total_de_votos'].sum(),
                                                         drama_depois_90['total_de_votos'].sum(),
                                                         acao_depois_90['total_de_votos'].sum(),
                                                         aventura_depois_90['total_de_votos'].sum(),
                                                         terror_depois_90['total_de_votos'].sum()] })
rank_de_votos_depois_90.sort_values('Total_de_votos',ascending=False)

A categoria **Drama** é a mais votada a partir da década de 1990, mas também os filmes de **ação** que estão em segundo estão com pouca diferença neste rank

In [ ]:
rank_de_votos_antes_90 = pd.DataFrame({'Categoria':['Comédia','Drama','Ação','Aventura','Terror'],
                                       'Total_de_votos':[comedia_antes_90['total_de_votos'].sum(),
                                                         drama_antes_90['total_de_votos'].sum(),
                                                         acao_antes_90['total_de_votos'].sum(),
                                                         aventura_antes_90['total_de_votos'].sum(),
                                                         terror_antes_90['total_de_votos'].sum()] })
rank_de_votos_antes_90.sort_values('Total_de_votos',ascending=False)

Já antes da década de 1990, a categoria **comédia** é estava na liderança, além disso, os filmes de **ação** também ficaram na segunda posião com uma pequena difernça.